# Final Training: Best Algorithm Extended Run

**Reinforcement Learning Summative Assignment - Final Model**

After completing all 4 algorithm notebooks (DQN, PPO, A2C, REINFORCE), this notebook trains the winning algorithm with extended timesteps and multiple seeds for production use.

## Workflow:
1. **Load Results**: Import summary JSONs from all 4 algorithms
2. **Compare**: Display side-by-side performance comparison
3. **Select Best**: Identify best algorithm + configuration
4. **Extended Training**: Train for 500K timesteps with 10 seeds
5. **Comprehensive Evaluation**: Test on 100 episodes
6. **Generate Demo Video**: Create visualization for report
7. **Export**: Save final model, results, and plots

**Estimated Runtime**: 8-12 hours on Colab GPU


## 1. Setup: Mount Google Drive & Install Dependencies


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set up project directory
import os
PROJECT_DIR = '/content/drive/MyDrive/RL_Summative'
os.makedirs(f'{PROJECT_DIR}/models/final', exist_ok=True)
os.makedirs(f'{PROJECT_DIR}/demos', exist_ok=True)

print(f"✓ Google Drive mounted")
print(f"✓ Project directory: {PROJECT_DIR}")


In [ ]:
# Install required packages
%pip install -q numpy==1.26.4
%pip install -q torch
%pip install -q gymnasium
%pip install -q stable-baselines3
%pip install -q sb3-contrib
%pip install -q matplotlib
%pip install -q seaborn
%pip install -q pandas
%pip install -q tqdm
%pip install -q imageio

print("\n" + "="*60)
print("✓ All packages installed!")
print("⚠️  RESTART RUNTIME if this is first install")
print("="*60)

In [ ]:
# Import libraries
import gymnasium as gym
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
from typing import Tuple, Dict, Any, Optional, List
from tqdm.notebook import tqdm
from gymnasium import spaces
from stable_baselines3 import PPO, A2C
from sb3_contrib import DQN
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries imported successfully!")


## 2. Load ClinicEnv (copy from DQN notebook)


In [ ]:
# Copy ClinicEnv class definition from DQN notebook cell 6
# (For brevity, not shown here - copy the entire class)

print("✓ ClinicEnv loaded - copy from DQN notebook")

## 3. Load All Algorithm Results


In [ ]:
# Load summaries from all algorithms
algorithms = ['dqn', 'ppo', 'a2c', 'reinforce']
summaries = {}

for alg in algorithms:
    path = f'{PROJECT_DIR}/results/{alg}_summary.json'
    if os.path.exists(path):
        with open(path, 'r') as f:
            summaries[alg] = json.load(f)
        print(f"✓ Loaded {alg.upper()} summary")
    else:
        print(f"⚠️  {alg.upper()} summary not found at {path}")

print(f"\n✓ Loaded {len(summaries)} algorithm summaries")

## 4. Compare All Algorithms


In [ ]:
# Create comparison table
comparison_data = []

for alg, summary in summaries.items():
    ft = summary.get('full_training', {})
    comparison_data.append({
        'Algorithm': alg.upper(),
        'Mean Reward': ft.get('mean_reward', 0),
        'Std Reward': ft.get('std_reward', 0),
        'Triage Accuracy (%)': ft.get('mean_triage_accuracy', 0),
        'Best Config': summary.get('best_config_id', 'unknown')
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Mean Reward', ascending=False)

print("\n" + "="*70)
print("ALGORITHM COMPARISON (Full Training Results)")
print("="*70)
print(comparison_df.to_string(index=False))
print("="*70)

# Identify winner
best_alg = comparison_df.iloc[0]['Algorithm'].lower()
print(f"\n🏆 WINNER: {best_alg.upper()}")
print(f"   Mean Reward: {comparison_df.iloc[0]['Mean Reward']:.2f}")
print(f"   Triage Accuracy: {comparison_df.iloc[0]['Triage Accuracy (%)']:.1f}%")

## 5. Extended Training with Best Algorithm

Train the winning algorithm for 500K timesteps with 10 different seeds for robustness.


In [ ]:
# Get best algorithm config
best_summary = summaries[best_alg]
best_config_id = best_summary['best_config_id']
best_config = best_summary['best_config']

print(f"\nBest Configuration: {best_config_id}")
print(f"Hyperparameters:")
for key, val in best_config.items():
    if key not in ['id', 'description']:
        print(f"  {key}: {val}")

# Load appropriate model class
if best_alg == 'dqn':
    from sb3_contrib import DQN as BestModel
elif best_alg == 'ppo':
    from stable_baselines3 import PPO as BestModel
elif best_alg == 'a2c':
    from stable_baselines3 import A2C as BestModel
else:
    print("NOTE: REINFORCE requires custom implementation - adapt training code")
    BestModel = None

print(f"\n✓ Model class loaded: {best_alg.upper()}")

In [ ]:
# Helper functions (copy from DQN notebook)
def evaluate_agent(model, env, num_episodes=50, deterministic=True):
    """Evaluate trained agent."""
    episode_rewards = []
    episode_lengths = []
    triage_accuracies = []
    
    for _ in range(num_episodes):
        obs, info = env.reset()
        done = False
        episode_reward = 0.0
        episode_length = 0
        correct = 0
        total = 0
        
        while not done:
            action, _ = model.predict(obs, deterministic=deterministic)
            obs, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            
            episode_reward += reward
            episode_length += 1
            
            if 'correct_action' in info:
                total += 1
                if action == info['correct_action']:
                    correct += 1
        
        episode_rewards.append(episode_reward)
        episode_lengths.append(episode_length)
        if total > 0:
            triage_accuracies.append(100.0 * correct / total)
    
    return {
        "mean_reward": np.mean(episode_rewards),
        "std_reward": np.std(episode_rewards),
        "mean_length": np.mean(episode_lengths),
        "mean_triage_accuracy": np.mean(triage_accuracies) if triage_accuracies else 0.0,
        "std_triage_accuracy": np.std(triage_accuracies) if triage_accuracies else 0.0
    }

print("✓ Helper functions defined")

In [ ]:
# Extended training
EXTENDED_SEEDS = [42, 123, 456, 789, 1024, 2048, 3072, 4096, 5120, 6144]
EXTENDED_TIMESTEPS = 500000

extended_results = []
extended_models = {}

print("\n" + "="*70)
print(f"EXTENDED TRAINING: {best_alg.upper()} × {len(EXTENDED_SEEDS)} seeds × 500K timesteps")
print("="*70)

for i, seed in enumerate(EXTENDED_SEEDS):
    print(f"\n[Seed {i+1}/{len(EXTENDED_SEEDS)}] Training with seed={seed}")
    print("-" * 70)
    
    env = ClinicEnv(seed=seed, max_steps=500)
    start_time = time.time()
    
    try:
        # Create model with best config
        model = BestModel(
            "MlpPolicy",
            env,
            **{k: v for k, v in best_config.items() if k not in ['id', 'description']},
            seed=seed,
            verbose=0
        )
        
        model.learn(total_timesteps=EXTENDED_TIMESTEPS)
        elapsed = time.time() - start_time
        
        # Evaluate
        eval_results = evaluate_agent(model, env, num_episodes=50)
        
        extended_results.append({
            "seed": seed,
            "mean_reward": eval_results["mean_reward"],
            "std_reward": eval_results["std_reward"],
            "triage_accuracy": eval_results["mean_triage_accuracy"],
            "training_time_sec": elapsed
        })
        
        extended_models[f"seed_{seed}"] = model
        
        print(f"✓ Completed in {elapsed/60:.1f} minutes")
        print(f"  Mean Reward: {eval_results['mean_reward']:.2f}")
        print(f"  Triage Accuracy: {eval_results['mean_triage_accuracy']:.1f}%")
        
        # Save
        model_path = f"{PROJECT_DIR}/models/final/{best_alg}_seed{seed}.zip"
        model.save(model_path)
        
    except Exception as e:
        print(f"✗ Error: {e}")
        continue
    
    env.close()

print("\n" + "="*70)
print("EXTENDED TRAINING COMPLETE!")
print("="*70)

## 6. Final Results & Analysis


In [ ]:
# Analyze extended training results
extended_df = pd.DataFrame(extended_results)

print("\n" + "="*70)
print(f"FINAL RESULTS: {best_alg.upper()} (Extended Training)")
print("="*70)
print(f"Total Timesteps: 500K × {len(EXTENDED_SEEDS)} seeds = {500000 * len(EXTENDED_SEEDS):,}")
print(f"Mean Reward: {extended_df['mean_reward'].mean():.2f} ± {extended_df['mean_reward'].std():.2f}")
print(f"Triage Accuracy: {extended_df['triage_accuracy'].mean():.1f}% ± {extended_df['triage_accuracy'].std():.1f}%")
print(f"Min Reward: {extended_df['mean_reward'].min():.2f}")
print(f"Max Reward: {extended_df['mean_reward'].max():.2f}")
print(f"Total Training Time: {extended_df['training_time_sec'].sum()/3600:.2f} hours")
print("="*70)

# Save results
extended_df.to_csv(f"{PROJECT_DIR}/results/final_extended_results.csv", index=False)
print(f"\n✓ Results saved to {PROJECT_DIR}/results/final_extended_results.csv")

In [ ]:
# Plot results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.bar(range(len(EXTENDED_SEEDS)), extended_df['mean_reward'], alpha=0.7, color='steelblue')
ax1.axhline(y=extended_df['mean_reward'].mean(), color='red', linestyle='--',
            label=f"Mean: {extended_df['mean_reward'].mean():.2f}")
ax1.set_xlabel('Seed Index', fontsize=12)
ax1.set_ylabel('Mean Reward', fontsize=12)
ax1.set_title(f'{best_alg.upper()} - Extended Training Rewards', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

ax2 = axes[1]
ax2.bar(range(len(EXTENDED_SEEDS)), extended_df['triage_accuracy'], alpha=0.7, color='coral')
ax2.axhline(y=extended_df['triage_accuracy'].mean(), color='red', linestyle='--',
            label=f"Mean: {extended_df['triage_accuracy'].mean():.1f}%")
ax2.set_xlabel('Seed Index', fontsize=12)
ax2.set_ylabel('Triage Accuracy (%)', fontsize=12)
ax2.set_title(f'{best_alg.upper()} - Triage Accuracy', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plot_path = f"{PROJECT_DIR}/plots/final_extended_training.png"
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
print(f"✓ Plot saved to {plot_path}")
plt.show()

## 7. Save Final Best Model


In [ ]:
# Select best model from extended training
best_seed_idx = extended_df['mean_reward'].idxmax()
best_seed = extended_df.loc[best_seed_idx, 'seed']
final_best_model = extended_models[f"seed_{best_seed}"]

final_model_path = f"{PROJECT_DIR}/models/FINAL_BEST_MODEL.zip"
final_best_model.save(final_model_path)

print("\n" + "="*70)
print("FINAL BEST MODEL")
print("="*70)
print(f"Algorithm: {best_alg.upper()}")
print(f"Config: {best_config_id}")
print(f"Seed: {best_seed}")
print(f"Mean Reward: {extended_df.loc[best_seed_idx, 'mean_reward']:.2f}")
print(f"Triage Accuracy: {extended_df.loc[best_seed_idx, 'triage_accuracy']:.1f}%")
print(f"Saved to: {final_model_path}")
print("="*70)

## 8. Create Final Summary


In [ ]:
# Create comprehensive final summary
final_summary = {
    "winning_algorithm": best_alg.upper(),
    "best_config_id": best_config_id,
    "best_config": best_config,
    "extended_training": {
        "num_seeds": len(EXTENDED_SEEDS),
        "timesteps_per_seed": EXTENDED_TIMESTEPS,
        "total_timesteps": len(EXTENDED_SEEDS) * EXTENDED_TIMESTEPS,
        "mean_reward": float(extended_df['mean_reward'].mean()),
        "std_reward": float(extended_df['mean_reward'].std()),
        "mean_triage_accuracy": float(extended_df['triage_accuracy'].mean()),
        "std_triage_accuracy": float(extended_df['triage_accuracy'].std()),
        "best_seed": int(best_seed),
        "best_seed_reward": float(extended_df.loc[best_seed_idx, 'mean_reward']),
        "total_training_time_hours": float(extended_df['training_time_sec'].sum() / 3600)
    },
    "all_algorithms_comparison": comparison_df.to_dict('records'),
    "final_model_path": final_model_path
}

summary_path = f"{PROJECT_DIR}/results/FINAL_SUMMARY.json"
with open(summary_path, 'w') as f:
    json.dump(final_summary, f, indent=2)

print("\n" + "="*70)
print("FINAL SUMMARY SAVED")
print("="*70)
print(json.dumps(final_summary, indent=2))
print("="*70)
print(f"\n✓ Summary saved to: {summary_path}")

## ✅ Training Complete!

**You now have:**
- ✅ Best algorithm identified
- ✅ Extensively trained model (5M timesteps)
- ✅ Comprehensive evaluation results
- ✅ All files saved to Google Drive

**Next Steps:**
1. Download final model and results from Google Drive
2. Create PDF report using the results and plots
3. Record 3-minute video demonstration
4. Submit to Canvas

**Files Location:**
- Models: `/content/drive/MyDrive/RL_Summative/models/`
- Results: `/content/drive/MyDrive/RL_Summative/results/`
- Plots: `/content/drive/MyDrive/RL_Summative/plots/`
